## Demonstration of the Item-Based Collaborative Recommender

This system leverages collaborative filtering by analyzing how user-item interactions bridges items
Therefore, it focuses on the user-item relation.

It recommends articles that these similar users have engaged with, aiming to provide personalized suggestions. The model's performance is evaluated using MAP@K and NDCG@K metrics.

In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

import polars as pl
import numpy as np

from parquet_data_reader import ParquetDataReader
from utils.data_preprocessing import DataProcesser
from models.collaborative.item_based_CF import ItemBasedCollaborativeRecommender

pl.Config.set_tbl_cols(-1)

polars.config.Config

## Data import & Preprocessing

In [2]:
data_reader = ParquetDataReader()
articles_df = data_reader.read_data('../../data/articles.parquet')
train_behaviors_df = data_reader.read_data('../../data/train/behaviors.parquet')
train_history_df = data_reader.read_data('../../data/train/history.parquet')
document_vectors_df = data_reader.read_data('../../data/document_vector.parquet')

We check the size of the data. From the sizes we learn that:
<ol>
  <li>We have 20738 unique articles</li>
  <li>We have 15143 unique users</li>
  <li>We have 232887 interactions in the testset</li>
</ol> 

In [3]:
print("articles_df has the size:         ", articles_df.shape)
print("train_behaviors_df has the size:  ", train_behaviors_df.shape)
print("train_history_df has the size:    ", train_history_df.shape)
print("document_vectors_df has the size: ", document_vectors_df.shape)

articles_df has the size:          (20738, 21)
train_behaviors_df has the size:   (232887, 17)
train_history_df has the size:     (15143, 5)
document_vectors_df has the size:  (125541, 2)


### Validation set

In [4]:
test_behaviours_df = data_reader.read_data('../../data/validation/behaviors.parquet')
test_behaviours_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
96791,null,2023-05-28 04:21:24,9.0,null,2,"[9783865, 9784591, … 9784710]",[9784696],22548,false,null,null,null,false,142,72.0,100.0
96798,null,2023-05-28 04:31:48,46.0,null,2,"[9782884, 9783865, … 9784648]",[9784281],22548,false,null,null,null,false,143,16.0,28.0
96801,null,2023-05-28 04:30:17,14.0,null,2,"[9784648, 7184889, … 9781983]",[9784444],22548,false,null,null,null,false,143,12.0,24.0
96808,null,2023-05-28 04:27:19,22.0,null,2,"[9784607, 9695098, … 9781983]",[9781983],22548,false,null,null,null,false,142,125.0,80.0
96810,null,2023-05-28 04:29:47,23.0,null,2,"[9781983, 7184889, … 9781520]",[9784642],22548,false,null,null,null,false,142,null,null


In [5]:
# Combine train and test behaviors
combined_df = pl.concat([train_behaviors_df, test_behaviours_df])

# Generate a random mask for splitting
n = combined_df.height  # Total number of rows
test_mask = np.random.rand(n) < 0.30  # 30% test, 70% train

# Apply the mask
test_behaviors_df = combined_df.filter(test_mask)
train_behaviors_df = combined_df.filter(~test_mask)

# Verify the split
print(f"Train size: {train_behaviors_df.shape[0]}, Test size: {test_behaviors_df.shape[0]}")

Train size: 334481, Test size: 143053


### Table contents

The information on news articles. As we are going to perform user-user CF, this table is not neccesary

In [6]:
articles_df.head()

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""
3012771,"""Morten Bruun fyret i Sønderjys…","""FODBOLD: Morten Bruun fyret me…",2023-06-29 06:20:39,false,"""Kemien mellem spillerne i Supe…",2006-05-01 14:28:40,[3177953],"""article_default""","""https://ekstrabladet.dk/sport/…",[],[],"[""Erhverv"", ""Kendt"", … ""Ansættelsesforhold""]",142,"[196, 199]","""sport""",null,null,null,0.8241,"""Negative"""
3023463,"""Luderne flytter på landet""","""I landets tyndest befolkede om…",2023-06-29 06:20:43,false,"""Det frække erhverv rykker på l…",2007-03-24 08:27:59,[3184029],"""article_default""","""https://ekstrabladet.dk/nyhede…",[],[],"[""Livsstil"", ""Erotik""]",118,[133],"""nyheder""",null,null,null,0.7053,"""Neutral"""
3032577,"""Cybersex: Hvornår er man utro?""","""En flirtende sms til den flott…",2023-06-29 06:20:46,false,"""De fleste af os mener, at et t…",2007-01-18 10:30:37,[3030463],"""article_default""","""https://ekstrabladet.dk/sex_og…",[],[],"[""Livsstil"", ""Partnerskab""]",565,[],"""sex_og_samliv""",null,null,null,0.9307,"""Neutral"""


Each file consists of seven days of impression logs. The train_behaviors_df table contains all information about interactions between users and items, and can be used as a basis for user-user CF. <strong>Therefore we only need this table</strong>.

In [7]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153078,9777492,2023-05-24 07:13:46,7.0,100.0,1,"[9778021, 9778627, … 7213923]",[9778226],151570,false,null,null,null,false,1976,4.0,21.0
155587,9778627,2023-05-24 07:56:30,50.0,100.0,1,"[9777397, 9759955, … 9778369]",[9778375],161621,false,null,null,null,false,3625,119.0,100.0
155588,9778769,2023-05-24 07:51:14,24.0,100.0,1,"[9775716, 9778745, … 9778718]",[9778718],161621,false,null,null,null,false,3625,59.0,100.0


Each file consists of users' click histories collected over 21 days period. This table does contain the same values as the train_behaviours_df, but as that table is easier to work with we will use train_behaviours_df over this one

In [8]:
train_history_df.head()

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"
20396,"[2023-04-27 12:30:44, 2023-04-27 12:31:34, … 2023-05-17 10:59:44]","[100.0, 59.0, … 13.0]","[9738760, 9738355, … 9769679]","[49.0, 34.0, … 4.0]"
34912,"[2023-04-29 07:12:49, 2023-04-29 13:01:18, … 2023-05-18 05:06:40]","[100.0, 35.0, … 27.0]","[9741802, 9741804, … 9770882]","[153.0, 7.0, … 5.0]"
37953,"[2023-04-27 19:17:10, 2023-04-27 19:17:27, … 2023-05-17 21:29:22]","[14.0, 28.0, … 18.0]","[9739205, 9739202, … 9769306]","[4.0, 16.0, … 6.0]"


List of vectors for each article. This is used to describe the items. It could be used for item-item CF, but is not relevant to user-user CF.  <strong>This table will therefore not be used</strong>

In [9]:
document_vectors_df.head()

article_id,document_vector
i32,list[f32]
3000022,"[0.065424, -0.047425, … 0.035706]"
3000063,"[0.028815, -0.000166, … 0.027167]"
3000613,"[0.037971, 0.033923, … 0.063961]"
3000700,"[0.046524, 0.002913, … 0.023423]"
3000840,"[0.014737, 0.024068, … 0.045991]"


From the analasys we see that we only need train_behaviour_df to perform user-user CF

## Preprocessing

### Remove non-needed values

We see that we have several items that are not required for performing user-user CF

In [10]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153078,9777492,2023-05-24 07:13:46,7.0,100.0,1,"[9778021, 9778627, … 7213923]",[9778226],151570,false,null,null,null,false,1976,4.0,21.0
155587,9778627,2023-05-24 07:56:30,50.0,100.0,1,"[9777397, 9759955, … 9778369]",[9778375],161621,false,null,null,null,false,3625,119.0,100.0
155588,9778769,2023-05-24 07:51:14,24.0,100.0,1,"[9775716, 9778745, … 9778718]",[9778718],161621,false,null,null,null,false,3625,59.0,100.0


All information that does not describe a user, or a user-item interaction can therefore be removed

In [11]:
irelevant_columns = ["impression_time", "device_type", "article_ids_inview", "article_ids_clicked", "session_id", "next_read_time", "next_scroll_percentage"]
train_behaviors_df = train_behaviors_df.drop(irelevant_columns)
train_behaviors_df.head()

impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,i32,f32,f32,u32,bool,i8,i8,i8,bool
149474,null,13.0,null,139836,false,null,null,null,false
153070,9777492,26.0,100.0,151570,false,null,null,null,false
153078,9777492,7.0,100.0,151570,false,null,null,null,false
155587,9778627,50.0,100.0,161621,false,null,null,null,false
155588,9778769,24.0,100.0,161621,false,null,null,null,false


The remaining items are the ones that can be used. But already here we see that we have several features with lacking information. We should therefore treat this

### Account for missing values

We see here that alot of the behaviours contain missing values. The therefore have to either remove or replace the values

In [12]:
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(334481, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,235225,0,236681,0,0,311507,327991,325600,0


In [13]:
train_behaviors_df = train_behaviors_df.filter(train_behaviors_df["article_id"].is_not_null())
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99256, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,2515,0,0,92445,97008,95846,0


We see that of 70421, are there between 65-68000 missing values for gender, postcode and age. We therefore remove these as there is no use subsidizing them

In [14]:
train_behaviors_df = train_behaviors_df.drop(["gender", "postcode", "age"])
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99256, 7)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,is_subscriber
u32,u32,u32,u32,u32,u32,u32
0,0,0,2515,0,0,0


We still see that 2570/98967 rows are missing a scroll percentage. As this is very low (<3%) we can easily replace this. Intitially we just set scroll to 0

In [15]:
train_behaviors_df = train_behaviors_df.fill_null(strategy="zero")

### Account for multiple instances of the same article and user

By checking rows where the user_id and article_id are the same we see that we have 9855 instances where the user has read the same article multiple times

In [16]:
duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)

print(duplicates)

shape: (12_359, 3)
┌────────────┬─────────┬───────┐
│ article_id ┆ user_id ┆ count │
│ ---        ┆ ---     ┆ ---   │
│ i32        ┆ u32     ┆ u32   │
╞════════════╪═════════╪═══════╡
│ 9773392    ┆ 2099328 ┆ 4     │
│ 9771916    ┆ 930790  ┆ 2     │
│ 9787332    ┆ 1138847 ┆ 2     │
│ 9781859    ┆ 2007832 ┆ 2     │
│ 9776238    ┆ 2227679 ┆ 7     │
│ …          ┆ …       ┆ …     │
│ 9781621    ┆ 2545456 ┆ 2     │
│ 9771796    ┆ 777735  ┆ 2     │
│ 9788797    ┆ 512125  ┆ 2     │
│ 9787501    ┆ 721001  ┆ 2     │
│ 9771352    ┆ 2010099 ┆ 2     │
└────────────┴─────────┴───────┘


C:\Users\magnu\AppData\Local\Temp\ipykernel_30332\4233841252.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)


We see that we need to combine these duplicate rows. We therefore propose that for multiple instances of the same article and user, we combine the readtime and select the largest scroll percentage. This way we can preserve the data without having duplicates

In [17]:
dataProcesser = DataProcesser()
behaviors_df = dataProcesser.collaborative_filtering_preprocess()
train_df, test_df = dataProcesser.random_split(behaviors_df, test_ratio=0.2)
print(train_df.head())

shape: (5, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9538375    ┆ 2390833 ┆ 946.0          ┆ 100.0      │
│ 9784870    ┆ 2477498 ┆ 32.0           ┆ 100.0      │
│ 9780983    ┆ 973352  ┆ 110.0          ┆ 100.0      │
│ 9777043    ┆ 1246193 ┆ 83.0           ┆ 100.0      │
│ 9785665    ┆ 1625478 ┆ 72.0           ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

This first model uses readtime and read percentage interactions to compare the user interactions 

In [18]:
recommender = ItemBasedCollaborativeRecommender(train_df)
recommender.fit()

{9538375: [(9660886, np.float64(0.769959261410969)),
  (9756546, np.float64(0.7699592614109689)),
  (9790019, np.float64(0.7324719672894666)),
  (9778318, np.float64(0.7114526632948414)),
  (9385951, np.float64(0.574827029703642)),
  (9655559, np.float64(0.43645421160306985)),
  (9780962, np.float64(0.3855355119772803)),
  (9653790, np.float64(0.3318899543152567)),
  (9765336, np.float64(0.3254397229689596)),
  (9790689, np.float64(0.32543192460630843))],
 9784870: [(9777406, np.float64(0.0017501859726339886)),
  (9789974, np.float64(0.0017302268671062215)),
  (9776303, np.float64(0.00159277859349094)),
  (9772284, np.float64(0.0015883838668356853)),
  (9782181, np.float64(0.0005786781733211344)),
  (9770997, np.float64(2.7507804592485918e-05)),
  (9779538, np.float64(2.750260375883329e-05)),
  (9771460, np.float64(2.599107677403456e-05)),
  (9774392, np.float64(2.116130326201837e-05)),
  (9786718, np.float64(1.1579775317627217e-05))],
 9780983: [(9782315, np.float64(0.2145413078500599

This first model just compares all artilces read by users when comparing users

In [19]:
binary_recommender = ItemBasedCollaborativeRecommender(train_df, binary_model=True)
binary_recommender.fit()

{9538375: [(9102576, np.float64(0.2581988897471611)),
  (9573677, np.float64(0.2581988897471611)),
  (9706270, np.float64(0.18257418583505536)),
  (9756546, np.float64(0.18257418583505536)),
  (9660886, np.float64(0.18257418583505536)),
  (9510890, np.float64(0.18257418583505536)),
  (9552181, np.float64(0.18257418583505536)),
  (9639962, np.float64(0.18257418583505536)),
  (9512930, np.float64(0.18257418583505536)),
  (9055748, np.float64(0.18257418583505536))],
 9784870: [(9785310, np.float64(0.0843235801592227)),
  (9783075, np.float64(0.08429826916355743)),
  (9566287, np.float64(0.08192319205190401)),
  (9500505, np.float64(0.08192319205190401)),
  (9736092, np.float64(0.08192319205190401)),
  (9679552, np.float64(0.08192319205190401)),
  (9445566, np.float64(0.08192319205190401)),
  (9155487, np.float64(0.08192319205190401)),
  (9730408, np.float64(0.08192319205190401)),
  (9645620, np.float64(0.08192319205190401))],
 9780983: [(9791082, np.float64(0.23145502494313785)),
  (96642

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model presentation

### Article Recommendation

In [20]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9776917, 9785596, 9777846, 9787261, 9782180]
reccomended for user  620796 :  [9782955, 9678305, 9700482, 9306867, 9779345]
reccomended for user  1067393 :  [9538375, 9784870, 9780983, 9777043, 9785665]
reccomended for user  1726258 :  []
reccomended for user  17205 :  [9766441, 9767955, 9774652, 9778917, 9482380]


In [21]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", binary_recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9774864, 9673979, 9756546, 9660886, 9717962]
reccomended for user  620796 :  [9765326, 9766178, 9749469, 9769197, 9512930]
reccomended for user  1067393 :  [9769471, 9097165, 9737266, 9575236, 9559619]
reccomended for user  1726258 :  []
reccomended for user  17205 :  [9768638, 9705425, 9768860, 8518755, 9766441]


### Evaluation Scores

#### Without ability to reccomend read articles

The complex model only reccomending articles the user has not yet read

In [22]:
results = recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.0025000000000000005),
 'NDCG@K': np.float64(0.002284133246515099)}

The binary reccomender model only reccomending articles the user has not yet read

In [23]:
results = binary_recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.006140350877192983),
 'NDCG@K': np.float64(0.022845994530473146)}

#### With ability to reccomend previously read articles

The complex model reccomending articles the user, even if they have read them before

In [24]:
results = recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.000925925925925926),
 'NDCG@K': np.float64(0.0006134650319850836)}

The binary recommender model reccomending articles the user, even if they have read them before

In [25]:
results = binary_recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.002061855670103093),
 'NDCG@K': np.float64(0.0073821945485884385)}

## Model Experimentation

In [26]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=100, allow_read_articles=True)
results = set(test_behaviours_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, 9782092, 9780815, None, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9776917, 9785596, 9777846, 9787261, 9782180, 4265340, 9789616, 9777969, 9790293, 9778413, 9771576, 9772710, 9775985, 9774764, 9759717, 9776691, 9778448, 9780193, 9783655, 9759345, 9772221, 9771355, 9770997, 9779538, 9776694, 9779724, 9774392, 9781859, 9772601, 9789502, 9764444, 9789404, 9788692, 9780791, 9779289, 9777464, 9772557, 9775323, 9778845, 9767903, 9714168, 9787022, 9676767, 9717962, 9772297, 9783276, 7086478, 9784869, 9778813, 9775283, 9789494, 9782915, 9771113, 9783952, 9722202, 9765759, 9777492, 9775568, 9771166, 9767233, 9788524, 9790548, 9781870, 9790414, 9773292, 9488208, 9774864, 9673979, 97

In [27]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=100, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9772045, 9781902, 9783824, 9774352, 9788705, 9756075, 9780020, 9769531, 9672256, 9778628, 9782092, 9783509, 9780181, 9776855, 9758424, 9786719, 9778413, 9776246, 9779577, 9780348}
[9776917, 9785596, 9777846, 9787261, 9782180, 4265340, 9789616, 9777969, 9790293, 9778413, 9771576, 9772710, 9775985, 9774764, 9759717, 9776691, 9778448, 9780193, 9783655, 9759345, 9772221, 9771355, 9770997, 9779538, 9776694, 9779724, 9774392, 9781859, 9772601, 9789502, 9764444, 9789404, 9788692, 9780791, 9779289, 9777464, 9772557, 9775323, 9778845, 9767903, 9714168, 9787022, 9676767, 9717962, 9772297, 9783276, 7086478, 9784869, 9778813, 9775283, 9789494, 9782915, 9771113, 9783952, 9722202, 9765759, 9777492, 9775568, 9771166, 9767233, 9788524, 9790548, 9781870, 9790414, 9773292, 9488208, 9774864, 9673979, 9780374, 9766803, 9660886, 9756546, 9578459, 9514605, 9769743, 9790755, 9773316, 9672256, 9711985, 9790019, 9778318, 9785668, 9774079, 9698332, 9783379, 9778745, 9771896, 9789479, 9759966, 9776386, 9385951,

#### Binary Item Based Recommender Metrics


In [30]:
from utils.evaluation import perform_model_evaluation
metrics = perform_model_evaluation(binary_recommender, test_data=test_df, k=5)
metrics

{'precision@k': np.float64(0.0034212401995723455),
 'recall@k': np.float64(0.009139394698910024),
 'fpr@k': np.float64(0.002172041467656416)}

In [33]:
from utils.evaluation import append_model_metrics
append_model_metrics(metrics, "item based binary")

#### Complex Item Based Recommender Metrics

In [34]:
metrics_complex = perform_model_evaluation(recommender, test_data=test_df, k=5)
print(metrics_complex)

append_model_metrics(metrics_complex, "item based complex")

{'precision@k': np.float64(0.002337847469707769), 'recall@k': np.float64(0.004269289735435344), 'fpr@k': np.float64(0.002174678001086813)}


### Diversity Evaluation
Calculates the aggrigate diversity of the recommender model recommendations, and appends the result to the `/evaluation_summary/model_overview_diversity.csv`-file. 

In [42]:
from utils.evaluation import aggregate_diversity
from utils.evaluation import append_aggregate_diversity

print(recommender.item_similarity_matrix)

diversity = aggregate_diversity(recommender, articles_df, user_sample=1000)

print("Diversity")
print(diversity)

append_aggregate_diversity(diversity, "item based CF")
#TODO: Should not be zero? 

{9538375: [(9660886, np.float64(0.769959261410969)), (9756546, np.float64(0.7699592614109689)), (9790019, np.float64(0.7324719672894666)), (9778318, np.float64(0.7114526632948414)), (9385951, np.float64(0.574827029703642)), (9655559, np.float64(0.43645421160306985)), (9780962, np.float64(0.3855355119772803)), (9653790, np.float64(0.3318899543152567)), (9765336, np.float64(0.3254397229689596)), (9790689, np.float64(0.32543192460630843))], 9784870: [(9777406, np.float64(0.0017501859726339886)), (9789974, np.float64(0.0017302268671062215)), (9776303, np.float64(0.00159277859349094)), (9772284, np.float64(0.0015883838668356853)), (9782181, np.float64(0.0005786781733211344)), (9770997, np.float64(2.7507804592485918e-05)), (9779538, np.float64(2.750260375883329e-05)), (9771460, np.float64(2.599107677403456e-05)), (9774392, np.float64(2.116130326201837e-05)), (9786718, np.float64(1.1579775317627217e-05))], 9780983: [(9782315, np.float64(0.21454130785005998)), (9790019, np.float64(0.0880512708

### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [ ]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(recommender, "item_based", user_id=test_user_id, n=5)
footprint

[codecarbon INFO @ 15:58:46] [setup] RAM Tracking...
[codecarbon INFO @ 15:58:46] [setup] CPU Tracking...


[codecarbon WARNING @ 15:58:46] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon WARNING @ 15:58:47] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700H but we don't know it. Please contact us.
[codecarbon INFO @ 15:58:47] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 15:58:47] [setup] GPU Tracking...
[codecarbon INFO @ 15:58:47] No GPU found.
[codecarbon INFO @ 15:58:47] >>> Tracker's metadata:
[codecarbon INFO @ 15:58:47]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 15:58:47]   Python version: 3.11.9
[codecarbon INFO @ 15:58:47]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 15:58:47]   Available RAM : 15.731 GB
[codecarbon INFO @ 15:58:47]   CPU count: 20
[codecarbon INFO @ 15:58:47]   CPU model: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 15:58:47]   GPU count: None
[codecarbon INFO @ 15:58:47]   GPU model: None
[codecarbon INFO @ 15:58:51] Saving emissions data to file c:\Users\magnu\NewDesk\An.sys\TDT4215\recommender_system\demostrations\output\item_based_fit_

{'fit': ({9787332: [(9780302, np.float64(0.9999999999979626)),
    (9781998, np.float64(0.9999999999979625)),
    (9773356, np.float64(0.9999999999979625)),
    (9788677, np.float64(0.9999999985231265)),
    (9779019, np.float64(0.9999999070770395)),
    (9779517, np.float64(0.9981137728424707)),
    (9782746, np.float64(0.9965881367910926)),
    (9790713, np.float64(0.04881853310523909)),
    (9788497, np.float64(0.017575705639713446)),
    (9780447, np.float64(0.01692479434677352))],
   9774079: [(8534547, np.float64(0.7038291926059211)),
    (9787465, np.float64(0.7038291926059179)),
    (9778845, np.float64(0.7021581420063797)),
    (9777164, np.float64(0.32184755741584015)),
    (9772629, np.float64(0.20315337662907296)),
    (9774549, np.float64(0.10648294706702999)),
    (9777299, np.float64(0.09038417617313599)),
    (9775388, np.float64(0.08928580921227969)),
    (9771340, np.float64(0.07664633038114)),
    (9790548, np.float64(0.07084655184372646))],
   9786359: [(9396917, np